In [95]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
sns.set()

In [96]:
%%capture
from tqdm.notebook import tqdm
tqdm.pandas()

In [97]:
rio = pd.read_csv('combined.csv')

In [98]:
cities = {'rio': rio, 'mumbai': mumbai, 'hyderabad': hyderabad, 'chennai': chennai, 'delhi': delhi}

### scale prices

In [99]:
ss = StandardScaler()
def scale_prices(prices):
    return ss.fit_transform(prices)

for city_name, city_df in cities.items():    
    city_df['price'] = scale_prices(city_df[['price']])  # scale price
    city_df['city'] = city_name                          # add 'city' label

### combine cities

In [100]:
combined = pd.DataFrame()
for city in cities.values():
    combined = pd.concat([combined, city], sort=False)

### bootstrap

In [101]:
def bootstrap(df):
    df.dropna(subset=['price', 'class'], inplace=True)
    return pd.concat([df, df[df['class'] == 1].sample(
                      df['class'].value_counts()[0] - df['class'].value_counts()[1], # calc n_samples
                      replace = True)]).reset_index().drop(columns='index')
combined = bootstrap(combined)
for city in cities.values():
    city = bootstrap(city)

# model

In [104]:
def ScoreXCity(city):
    # define train XY
    train_df = combined[combined['city']!=city]
    x_train = train_df[['price']]
    y_train = train_df['class']
    
    # define test XY
    test_df = cities[city]
    x_test = test_df[['price']]
    y_test = test_df['class']
    
    # fit and score RF
    rf = RandomForestClassifier(n_estimators=20)
    rf.fit(x_train, y_train)
    return round(rf.score(x_train, y_train),4), round(rf.score(x_test, y_test),4)

In [105]:
# run x-tests
for city in cities.keys():
    train_score, test_score = ScoreXCity(city)
    print(f'Test on {city}: {test_score}')

Test on rio: 0.8208
Test on mumbai: 0.7772
Test on hyderabad: 0.864
Test on chennai: 0.7543
Test on delhi: 0.6418


In [53]:
combined = combined[['price', 'class']]

In [54]:
combined.to_csv('COMBINED.csv', index=False)